In [0]:
#pip install pandas
#pip install numpy
#pip install openpyxl
#pip install datetime
#pip install pyspark

# TR**ATE OS DADOS**

### **1º Passo:** Leia os dados fornecidos em csv e atribua-os a um dataframe.

*Após ler a base, analise-a para um reconhecimento inicial, você perceberá que ela está em um formato mensal, ou seja, em cada mês existe um registro do colaborador apresentando os dados dele respectivamente.*

### **2º Passo:** Trate os dados garantindo que os critérios abaixo sejam cumpridos:

1. **Completude**: em nossas bases, em cada mês, colunas como data de admissão, data de nascimento, id do funcionário, grau de instrução, cargo e salário não podem conter valores nulos.
2. **Unicidade**: colunas que são chaves não podem se repetir dentro de um mês referência, o que quer dizer que os dados de um funcionário não podem aparecer mais de uma vez dentro de um mês.
3. **Integridade**: temos que garantir que em determinada coluna haja valores condizentes ao tipo e à finalidade dela. Por exemplo: se a coluna é do tipo texto, não pode haver valores incompatíveis como, por exemplo, números. Garanta a integridade dos dados em cada coluna.
4. **Padronização de categorias**: colunas de categorias, não podem conter outros valores além das categorias previstas para ela.
5. **Padronize os valores dentro das colunas**: 
    1. Valores do tipo texto devem ter a primeira letra de uma palavra no modo maiúsculo;
    2. Remova espaços antes e depois de todas as colunas do tipo string;
    3. Colunas de data devem estar no formato date;
    4. Salário deve estar no formato double ou float.

### **3º Passo:** Crie novas colunas conforme se pede abaixo:

1. **Admitido no mês**: essa coluna deverá ser do tipo numérico, respeitando a seguinte condição: se a data de admissão do registro for igual ao mês de referência analisado, admitido no mês recebe valor é igual a 1, senão recebe valor igual a 0.
2. **Demitido no mês**: essa coluna deverá ser do tipo numérico, respeitando a seguinte condição: se a data de demissão do registro for igual ao mês de referência analisado, demitido no mês recebe valor é igual a 1, senão recebe valor igual a 0.
3. **Idade**: calcule a idade em anos no mês de referência em relação à data de nascimento.
4. **Escolaridade categorizada**: com base na coluna grau de instrução, categorize a escolaridade do colaborador para conter apenas as seguintes categorias: ensino fundamental incompleto, ensino fundamental completo, ensino médio incompleto, ensino médio completo, ensino superior incompleto, ensino superior completo, pós graduação, mestrado, doutorado.
5. **Tempo de empresa**: com base no mês referência e a data de admissão, obtenha o tempo de empresa do colaborador em meses.
6. **Mudou de salário**: compare o salário do mês atual com o salário do mês anterior dos funcionários, se houve mudança atribua 1 como valor para a coluna do mês atual, senão atribua 0.
7. **Mudou de cargo**: compare o cargo do mês atual com o cargo do mês anterior dos funcionários, se houve mudança atribua 1 como valor para a coluna do mês atual, senão atribua 0.

NOTA PESSOAL \
É minha primeira vez utilizando o pyspark, me adaptei como pude mas por fim acabei utilizando o API do pandas pro pyspark.

Tive de importar inumeros pacotes pois estava dando constantes erros durante a limpeza do arquivo.

In [0]:
import pandas as pd
import numpy as np
import pyspark.pandas as ps
import openpyxl
import pyspark.sql 
import datetime as dt
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import warnings


Para elminar de vez logo os itens com o formato incorreto preferi montar uma structype e remover o salario errado

In [0]:
from pyspark.sql.types import *

sch=StructType([ 
    StructField("mes_referencia",StringType(),False),
    StructField("id_funcionario",IntegerType(),True),
    StructField("data_de_nascimento",StringType(),True),
    StructField("data_de_admissao",StringType(),True),
    StructField("data_de_demissao",StringType(),True),
    StructField("grau_de_instrucao",StringType(),True),
    StructField("cargo",StringType(),True),
    StructField("salario",FloatType(),True),
]) 


df = spark \
    .read.format("csv")\
    .option("header", "True")\
    .option("delimiter", ";")\
    .schema(sch) \
    .csv(arquivo)

arquivo = ("dbfs:/FileStore/shared_uploads/eziogb@gmail.com/base_mensalizada_de_funcionarios-2.csv")


#sch=StructType([ 
#    StructField("mes_referencia",StringType(),True),
#    StructField("id_funcionario",IntegerType(),True),
#    StructField("data_de_nascimento",StringType(),True),
#    StructField("data_de_admissao",StringType(),True),
#    StructField("data_de_demissao",StringType(),True),
#    StructField("grau_de_instrucao",StringType(),True),
#    StructField("cargo",StringType(),True),
#    StructField("salario",FloatType(),True),
#]) 





Criei uma df como psdf somente para realizar a limpeza de alguns dados e consequentemente realizar as frames que iria dar join

In [0]:
psdf = df

In [0]:
psdf = psdf.dropna(subset = ['data_de_admissao'])
psdf = psdf.dropna(subset = ['grau_de_instrucao'])
psdf = psdf.dropna(subset = ['salario'])
psdf = psdf.dropna(subset = ['cargo'])
psdf = psdf.drop_duplicates(subset = ['id_funcionario'])

Frame usuário, limitar o escopo e remover duplicidade de dados únicos.

In [0]:
funcionarios = psdf
funcionarios = funcionarios.drop('mes_referencia', 'data_de_demissao', 'grau_de_instrucao', 'cargo', 'salario')
funcionarios = funcionarios.withColumnRenamed('id_funcionario','id_func')

df dados, literalamente os dados a serem analisados.

In [0]:
dadosx = df
# Devido não saber o salário do funcionário no mês e sendo manipulação indevida de dados definir um valor, foi removido a linha onde o salário de um colaborador está como um texto.
dadosx = dadosx.filter(dadosx['salario'] > 0)
dados = dadosx
dados = dados.drop('data_de_nascimento', 'data_de_admissao')

realizando Join e conversando a tabela para pandas;

In [0]:
tabela = dados.join(funcionarios, dados.id_funcionario == funcionarios.id_func, 'inner')
tabela = tabela.drop('id_func')

tabela2 = tabela.to_pandas_on_spark()

Realizando classificação categorizada.

In [0]:
tabela2.loc[tabela2['grau_de_instrucao'] == "4ª série completo", 'grau_de_instrucao'] = "ensino fundamental completo"
tabela2.loc[tabela2['grau_de_instrucao'] == "ensino médio comp", 'grau_de_instrucao'] = "ensino medio completo"
tabela2.loc[tabela2['grau_de_instrucao'] == "gerente de produção", 'grau_de_instrucao'] = "gerente de producao"
tabela2.loc[tabela2['grau_de_instrucao'] == "líder de tecnologia", 'grau_de_instrucao'] = "lider de tecnologia"

Por algum motivo o pyspark estava retornando "null" em todas as "DateType", com isso eu não estava conseguindo realizar operações nessas colunas, para resolver esse problema foi necessário um modo criativo, onde transformei as datas em str em outras colunas e realizei calculos entre elas e por fim as juntei novamente.

In [0]:
tabela2['mes_referencia'] = ps.to_datetime(tabela2['mes_referencia'])
tabela2['mes_referencia'] = tabela2['mes_referencia'].dt.strftime('%m/%d/%Y')
tabela2['data_de_admissao'] = ps.to_datetime(tabela2['data_de_admissao'])
tabela2['data_de_admissao'] = tabela2['data_de_admissao'].dt.strftime('%d/%m/%Y')
tabela2['data_de_demissao'] = ps.to_datetime(tabela2['data_de_demissao'])
tabela2['data_de_demissao'] = tabela2['data_de_demissao'].dt.strftime('%d/%m/%Y')
tabela2['data_de_nascimento'] = ps.to_datetime(tabela2['data_de_nascimento'])
tabela2['data_de_nascimento'] = tabela2['data_de_nascimento'].dt.strftime('%d/%m/%Y')
tabela2['mes_referencia2'] = tabela2['mes_referencia']
tabela2['data_de_admissao2'] = tabela2['data_de_admissao']
tabela2['data_de_demissao2'] = tabela2['data_de_demissao']
tabela2['data_de_nascimento2'] = tabela2['data_de_nascimento']
tabela2[['MesRefX', 'DiaRefX', 'AnoRefX']] = tabela2['mes_referencia2'].str.split("/", expand=True, n=2)
tabela2['mes_ref'] = tabela2['DiaRefX'] + "/" + tabela2['MesRefX'] + "/" + tabela2['AnoRefX']
tabela2['MesRefX'] = tabela2['MesRefX'].astype(int)
tabela2['AnoRefX'] = tabela2['AnoRefX'].astype(int)
tabela2[['DiaNascX', 'MesNacX', 'AnoNascX']] = tabela2['data_de_nascimento2'].str.split("/", expand=True, n=2)
tabela2['MesNacX'] = tabela2['MesNacX'].astype(int)
tabela2['AnoNascX'] = tabela2['AnoNascX'].astype(int)
tabela2['Idade(Em anos)'] = (((tabela2['AnoRefX'] - tabela2['AnoNascX'])*365) + ((tabela2['MesRefX'] - tabela2['MesNacX'])*30))/365
tabela2['Idade(Em anos)'] = tabela2['Idade(Em anos)'].astype(int)
tabela2[['DiaAdmX', 'MesAdmX', 'AnoAdmX']] = tabela2['data_de_admissao2'].str.split("/", expand=True, n=2)
tabela2['MesAdmX'] = tabela2['MesAdmX'].astype(int)
tabela2['AnoAdmX'] = tabela2['AnoAdmX'].astype(int)
tabela2['Tempo_de_empresa(Em meses)'] = (((tabela2['AnoRefX'] - tabela2['AnoAdmX'])*365) + ((tabela2['MesRefX'] - tabela2['MesAdmX'])*30))/30
tabela2['Tempo_de_empresa(Em meses)'] = tabela2['Tempo_de_empresa(Em meses)'].astype(int)
tabela2.loc[tabela2['Tempo_de_empresa(Em meses)'] < 0, 'Tempo_de_empresa(Em meses)'] = tabela2['Tempo_de_empresa(Em meses)']*-1
tabela2[['DiaDem', 'MesDem', 'AnoDem']] = tabela2['data_de_demissao2'].str.split("/", expand=True, n=2)
tabela2['MesDem'] = tabela2['MesDem'].astype(int)
tabela2['AnoDem'] = tabela2['AnoDem'].astype(int)
tabela2['MesAdmC'] = (tabela2['MesRefX'] - tabela2['MesAdmX'])*30
tabela2['AnoAdmC'] = (tabela2['AnoRefX'] - tabela2['AnoAdmX'])*365
tabela2['Admitido_no_mesX'] = tabela2['MesAdmC'] + tabela2['AnoAdmC']
tabela2.loc[tabela2['Admitido_no_mesX'] == "0", 'Admitido_no_mes'] = int(1)
tabela2.loc[tabela2['Admitido_no_mesX'] != "0", 'Admitido_no_mes'] = int(0)
tabela2['MesDemC'] = (tabela2['MesRefX'] - tabela2['MesDem'])*30
tabela2['AnoDemC'] = (tabela2['AnoRefX'] - tabela2['AnoDem'])*365
tabela2['Demitido_no_mesX'] = tabela2['MesAdmC'] + tabela2['AnoAdmC']
tabela2.loc[tabela2['Demitido_no_mesX'] == 0, 'Demitido_no_mes'] = int(1)
tabela2.loc[tabela2['Demitido_no_mesX'] != 0, 'Demitido_no_mes'] = int(0)
tabela2 = tabela2.drop(['DiaAdm', 'DiaDem', 'MesAnoDem', 'DiaRef', 'DiaRefX', 'MesRefX', 'AnoRefX', 'DiaNascX', 'MesNacX', 'AnoNascX', 'mes_referencia2', 'data_de_admissao2', 'data_de_demissao2', 'data_de_nascimento2', 'DiaAdmX', 'MesAdmX', 'AnoAdmX', 'MesAnoRef', 'MesAnoAdm', 'Demitido_no_mesX', 'Admitido_no_mesX', 'MesDemC', 'AnoDemC', 'MesDem', 'AnoDem', 'MesAdmC', 'AnoAdmC'])

Continuação da parte de cima, porém focado em já realizar o campo de salário mês anterior

In [0]:
tabela2[['DiaRefX', 'MesRefX', 'AnoRefX']] = tabela2['mes_referencia'].str.split("/", expand=True, n=2)
tabela2['MesRefX'] = tabela2['MesRefX'].astype(int)
tabela2['MesRefX'] = tabela2['MesRefX'] - 1
tabela2['MesRefX'] = tabela2['MesRefX'].astype(str)
tabela2.loc[tabela2['MesRefX'] == 0, 'MesRefX'] = 12
tabela2.loc[tabela2['MesRefX'] < 10, 'MesRefX'] = "0"+tabela2['MesRefX']
tabela2['AnoRefX'] = tabela2['AnoRefX'].astype(int)
tabela2.loc[tabela2['MesRefX'] == 12, 'AnoRefX'] = tabela2['AnoRefX']-1
tabela2['AnoRefX'] = tabela2['AnoRefX'].astype(str)
tabela2['mes_ref'] = tabela2['DiaRefX'] + "/" + tabela2['MesRefX'] + "/" + tabela2['AnoRefX'] 
tabela2 = tabela2.drop(['DiaRefX', 'MesRefX', 'AnoRefX'])

df salário, foi necessário a elaboração de um novo dataframe para dar join diretamente a uma coluna a qual representava o mês anterior no mês atual, com isso o retorno seria o salário do mês anterior.

In [0]:
ss = tabela2
ss = ss.drop(['data_de_demissao', 'grau_de_instrucao', 'cargo', 'data_de_nascimento', 'data_de_admissao', 'mes_ref', 'Idade(Em anos)', 'Tempo_de_empresa(Em meses)', 'Admitido_no_mes', 'Demitido_no_mes', 'DiaRefX', 'MesRefX', 'AnoRefX'])
ss = ss.rename({'mes_referencia': 'mes_ref'}, axis=1)
tabela3 = ps.merge(tabela2, ss, on=['mes_ref', 'id_funcionario'], how='left')

Mesmo esquema da parte de cima, porém focado em cargo.

In [0]:
ca = tabela2
ca = ca.drop(['data_de_demissao', 'grau_de_instrucao', 'salario', 'data_de_nascimento', 'data_de_admissao', 'mes_ref', 'Idade(Em anos)', 'Tempo_de_empresa(Em meses)', 'Admitido_no_mes', 'Demitido_no_mes', 'DiaRefX', 'MesRefX', 'AnoRefX'])
ca = ca.rename({'mes_referencia': 'mes_ref'}, axis=1)
tabela3 = ps.merge(tabela3, ca, on=['mes_ref', 'id_funcionario'], how='left')

Retornando se o salário e o cargo mudaram.

In [0]:
tabela3.fillna(value=0, inplace=True)
tabela3.loc[tabela3['salario_y'] == 0, 'salario_y'] = tabela3['salario_x']
tabela3.loc[tabela3['cargo_y'] == 0, 'cargo_y'] = tabela3['cargo_x']
tabela3.loc[tabela3['salario_x'] == tabela3['salario_y'], 'Recebeu_aumento'] = 0
tabela3.loc[tabela3['salario_x'] != tabela3['salario_y'], 'Recebeu_aumento'] = 1
tabela3.loc[tabela3['cargo_x'] == tabela3['cargo_y'], 'Foi_promovido'] = 0
tabela3.loc[tabela3['cargo_x'] != tabela3['cargo_y'], 'Foi_promovido'] = 1
tabela3 = tabela3.drop(['cargo_y', 'salario_y', 'mes_ref'])

Removi duplicatas novamente pois após os join a planilha criou algumas linhas a mais.

In [0]:
tabela3 = tabela3.drop_duplicates(subset = ['id_funcionario', 'mes_referencia', 'salario_x', 'cargo_x'])

In [0]:
tabela3['cargo_x'] = tabela3['cargo_x'].str.capitalize()
tabela3['grau_de_instrucao'] = tabela3['grau_de_instrucao'].str.capitalize()

In [0]:
tabela3['mes_referencia'] = tabela3['mes_referencia'].astype(str)
#tabela3['data_de_demissao'] = tabela3['data_de_demissao'].astype(str)
tabela3['data_de_nascimento'] = tabela3['data_de_nascimento'].astype(str)
tabela3['data_de_admissao'] = tabela3['data_de_admissao'].astype(str)
tabela3['mes_referencia'] = ps.to_datetime(tabela3['mes_referencia'])
#tabela3['data_de_demissao'] = ps.to_datetime(tabela3['data_de_demissao'])
tabela3['data_de_nascimento'] = ps.to_datetime(tabela3['data_de_nascimento'])
tabela3['data_de_admissao'] = ps.to_datetime(tabela3['data_de_admissao'])

Afim de atender todos os critérios no teste fui obrigado a retornar os dados para "datetype", porém como havia informado anteriormente, por algum motivo esse formato está dando algum erro, e não consegui formata-lo no modelo correto de dd/mm/aaaa.
Fiz o melhor que pude com o que possuia em mãos e em todo o pai google de conhecimento.
Espero que por mais que esse tenha sido um pouco complicado pela minha falta de conhecimento, os próximos junto de vocês possa ser bem melhores!

Ps.: Eu fiz a mesma extração no pandas pelo Jupyter e demorei só alguns minutinhos.

In [0]:
tabela3.display()

mes_referencia,id_funcionario,data_de_demissao,grau_de_instrucao,cargo_x,salario_x,data_de_nascimento,data_de_admissao,Idade(Em anos),Tempo_de_empresa(Em meses),Admitido_no_mes,Demitido_no_mes,Recebeu_aumento,Foi_promovido
2020-01-01T00:00:00.000+0000,11223,0,Ensino fundamental completo,Operador de colheita i,1500.0,1995-04-01T00:00:00.000+0000,2018-06-01T00:00:00.000+0000,25,24,0,0,0,0
2020-01-02T00:00:00.000+0000,11223,0,Ensino fundamental completo,Operador de colheita i,1500.0,1995-04-01T00:00:00.000+0000,2018-06-01T00:00:00.000+0000,25,24,0,0,0,0
2020-01-03T00:00:00.000+0000,11223,0,Ensino fundamental completo,Operador de colheita i,1500.0,1995-04-01T00:00:00.000+0000,2018-06-01T00:00:00.000+0000,25,24,0,0,0,0
2020-01-04T00:00:00.000+0000,11223,0,Ensino fundamental completo,Operador de colheita i,1500.0,1995-04-01T00:00:00.000+0000,2018-06-01T00:00:00.000+0000,25,24,0,0,0,0
2020-01-05T00:00:00.000+0000,11223,0,Ensino fundamental completo,Operador de colheita i,1500.0,1995-04-01T00:00:00.000+0000,2018-06-01T00:00:00.000+0000,25,24,0,0,0,0
2020-01-06T00:00:00.000+0000,11223,0,Ensino fundamental completo,Operador de colheita i,1500.0,1995-04-01T00:00:00.000+0000,2018-06-01T00:00:00.000+0000,25,24,0,0,0,0
2020-01-07T00:00:00.000+0000,11223,0,Ensino fundamental completo,Operador de colheita i,1500.0,1995-04-01T00:00:00.000+0000,2018-06-01T00:00:00.000+0000,25,24,0,0,0,0
2020-01-08T00:00:00.000+0000,11223,0,Ensino fundamental completo,Operador de colheita i,1500.0,1995-04-01T00:00:00.000+0000,2018-06-01T00:00:00.000+0000,25,24,0,0,0,0
2020-01-09T00:00:00.000+0000,11223,0,Ensino fundamental completo,Operador de colheita ii,1750.0,1995-04-01T00:00:00.000+0000,2018-06-01T00:00:00.000+0000,25,24,0,0,1,1
2020-01-10T00:00:00.000+0000,11223,0,Ensino fundamental completo,Operador de colheita ii,1750.0,1995-04-01T00:00:00.000+0000,2018-06-01T00:00:00.000+0000,25,24,0,0,0,0


In [0]:
tabela3.info()

<class 'pyspark.pandas.frame.DataFrame'>
Int64Index: 114 entries, 0 to 136
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype     
---  ------                      --------------  -----     
 0   mes_referencia              114 non-null    datetime64
 1   id_funcionario              114 non-null    int32     
 2   data_de_demissao            114 non-null    object    
 3   grau_de_instrucao           114 non-null    <U0       
 4   cargo_x                     114 non-null    <U0       
 5   salario_x                   114 non-null    float32   
 6   data_de_nascimento          114 non-null    datetime64
 7   data_de_admissao            114 non-null    datetime64
 8   Idade(Em anos)              114 non-null    int64     
 9   Tempo_de_empresa(Em meses)  114 non-null    int64     
 10  Admitido_no_mes             114 non-null    int32     
 11  Demitido_no_mes             114 non-null    int32     
 12  Recebeu_aumento             114 non-null    int